In [217]:
import numpy as np
from scipy import special

In [218]:
class NeuralNetwork:
    """
    A simple implementation of a Neural Network.

    Attributes:
        inodes (int): The number of input nodes.
        hnodes (int): The number of hidden nodes.
        onodes (int): The number of output nodes.
        wih (numpy.ndarray): The weights from the input layer to the hidden layer.
        who (numpy.ndarray): The weights from the hidden layer to the output layer.
        lr (float): The learning rate for the neural network.
        activation_function (function): The activation function for the neural network.

    🤖 prompt: generate docs
    """
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # Weight sampling rule of thumb (page 103 book DLO): 
        #   - normal distribution: mean zero
        #   - standard deviation: inverse of the square root of the number of links into a node
        self.wih = np.random.normal(0.0, pow(self.hnodes,0.5), (self.hnodes, self.inodes))  # weights from input to hidden layer
        self.who = np.random.normal(0.0, pow(self.onodes,0.5), (self.onodes, self.hnodes))  # weights from hidden to output layer
        
        self.lr = learningrate
        self.activation_function = lambda x: special.expit(x)   # sigmoid function

    def forward(self, inputs_list):
        """
        Performs the forward pass of the neural network.

        Args:
            inputs_list (list): A list of input values for a single batch of training examples.

        Returns:
            tuple: A tuple containing the inputs, hidden outputs, and final outputs as numpy arrays.

        🤖 prompt: generate docs
        """
        inputs = np.array(inputs_list, ndmin=2).T
        hidden_inputs = self.wih @ inputs                           # calculate signals into hidden layer, using matrix multiplication
        hidden_outputs = self.activation_function(hidden_inputs)    # pass through activation function to calculate final values
        final_inputs = self.who @ hidden_outputs                    # calculate signals into output layer, using matrix multiplication
        final_outputs = self.activation_function(final_inputs)      # pass through activation function to calculate final values
        return inputs, hidden_outputs, final_outputs

    def backward(self, targets_list, final_outputs, hidden_outputs, inputs):
        """
        Performs the backward pass of the neural network, updating the weights based on the error of the output.

        Args:
            targets_list (list): A list of target values for a single batch of training examples.
            final_outputs (numpy.ndarray): The final output values from the forward pass of the neural network.
            hidden_outputs (numpy.ndarray): The hidden layer output values from the forward pass of the neural network.
            inputs (numpy.ndarray): The input values for a single batch of training examples.

        Returns:
            None

        🤖 prompt: generate docs
        """
        targets = np.array(targets_list, ndmin=2).T
        output_errors = targets - final_outputs     # output layer is the final layer, so error is (target - actual)
        hidden_errors = self.who.T @ output_errors  # errors propagated to the hidden layer

        # calculate the gradient of the activation function (sigmoid)
        self.who += self.lr * (output_errors * final_outputs * (1.0 - final_outputs)) @ hidden_outputs.T    # update the weights for the links between the hidden and output layers
        self.wih += self.lr * (hidden_errors * hidden_outputs * (1.0 - hidden_outputs)) @ inputs.T          # update the weights for the links between the input and hidden layers
        pass

    def train(self, inputs_list, targets_list):
        """
        Trains the neural network on a single batch of inputs and targets.

        Args:
            inputs_list (list): A list of input values for a single batch of training examples.
            targets_list (list): A list of target values for a single batch of training examples.

        Returns:    
            None

        🤖 prompt: generate docs
        """
        inputs, hidden_outputs, final_outputs = self.forward(inputs_list)   # forward pass
        self.backward(targets_list, final_outputs, hidden_outputs, inputs)  # backward pass

    def query(self, inputs_list):
        """
        Given an input list, returns the output of the neural network.

        Args:
            inputs_list (list): A list of input values for the neural network.

        Returns:
            numpy.ndarray: A 2D numpy array containing the output values of the neural network.

        🤖 prompt: generate docs
        """
        _, _, final_outputs = self.forward(inputs_list)     # forward pass
        return final_outputs

In [219]:
input_nodes = 3
hidden_nodes = 1
output_nodes = 3
learning_rate = 0.1

nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

# In supervised learning, we would normally use many input-target pairs
# and use each pair to update the network weights.
inputs = [1.0, 0.5, -1.5]
targets = [0.5, 0.5, 0.5]
nn.train(inputs, targets)

outputs = nn.query(inputs)
outputs

array([[0.48942926],
       [0.48160294],
       [0.59299874]])